## Read and tokenize data

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

#raw_datasets = load_dataset("glue", "mrpc")
raw_datasets = load_dataset('csv', data_files={'train': ['train.csv'], 'eval': 'test.csv'})


# checkpoint = "bert-base-uncased"
checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Using custom data configuration default-d8f4c168f3d8923e
Reusing dataset csv (/users/phd/kpawan/.cache/huggingface/datasets/csv/default-d8f4c168f3d8923e/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [2]:
tokenized_datasets["train"].column_names

['attention_mask', 'input_ids', 'label', 'text']

In [3]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names


['attention_mask', 'input_ids', 'labels']

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["eval"], batch_size=8, collate_fn=data_collator
)


## Test batches

In [5]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([32, 75]),
 'input_ids': torch.Size([32, 75]),
 'labels': torch.Size([32])}

##  Prepare and Train model

In [6]:
import torch
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
from tqdm import tqdm

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=19)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

## Evaluate

In [7]:
from datasets import load_metric

metric = load_metric("accuracy")
model.eval()
y_pred = []
y_true = []
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
    y_pred.extend(predictions.cpu().detach().numpy())
    y_true.extend(batch["labels"].cpu().detach().numpy())
    
metric.compute()

{'accuracy': 0.853882959146117}

In [8]:
from sklearn.metrics import f1_score, confusion_matrix
print(f1_score(y_true, y_pred, average='micro'))
confusion_matrix(y_true, y_pred)

0.8538829591461171


array([[ 46,   0,   0,   0,   1,   0,   0,   0,   0,   1,   0,   1,   0,
          0,   0,   1,   1,   0,   0],
       [  0,  43,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   3,   1,   0],
       [  0,   0, 105,   0,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   3,  23,   0,   0],
       [  0,   0,   0, 181,   0,   3,   0,   0,   0,   2,   0,   0,   0,
          0,   0,   0,   7,   1,   0],
       [  0,   0,   5,   0, 131,   0,   2,   0,   0,   0,   0,   1,   0,
          3,   3,   0,   5,   0,   0],
       [  0,   0,   0,   4,   0,  95,   0,   2,   0,   4,   0,   0,   0,
          0,   0,   2,   1,   0,   0],
       [  0,   1,   0,   0,   1,   0,  35,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   2],
       [  0,   0,   0,   0,   0,   1,   0,  19,   0,   0,   0,   0,   0,
          0,   0,   0,   2,   0,   0],
       [  0,   0,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0